In [5]:
import requests
import pandas as pd
import zipfile
import io
import os

import xml.etree.ElementTree as ET
import html

import numpy as np
os.makedirs("results/inputs_xml", exist_ok=True)
# ALL YEARS (consuming 0.37 GO)
years = ['2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024', '2025']
# ONE YEARS:
#years = ['2007']

In [6]:
for year in years:
    print("LOAD", year)
    ###### LOAD DATAS TO LOCAL XML TO DF ##########
    
#     # Source = "https://www.prix-carburants.gouv.fr/rubrique/opendata/" (site gouvernemental)
#     url = f"https://donnees.roulez-eco.fr/opendata/annee/{year}"
    
#     response = requests.get(url)
#     print(response)
    
#     # get filename
#     zip_file = zipfile.ZipFile(io.BytesIO(response.content))
#     file_name = zip_file.namelist()[0]
#     print(file_name)
    
#     # get original xml file and save in local
#     with zipfile.ZipFile(io.BytesIO(response.content), 'r') as zip_ref:
#         zip_ref.extractall("results/inputs_xml")
#     print(file_name, "extracted")
    
    ###### TRANSFORM LOCAL XML TO DF ##########
    
    # transform xml to df
    tree = ET.parse(f"results/inputs_xml/PrixCarburants_annuel_{year}.xml")
    root = tree.getroot()
    data = []
    for pdv in root.findall("pdv"):
        for p in pdv.findall("prix"):
            row = {
                "id": pdv.get("id"),
                "latitude": pdv.get("latitude"),
                "longitude": pdv.get("longitude"),
                "cp": pdv.get("cp"),
                "ville": pdv.find("ville").text if pdv.find("ville") is not None and pdv.find("ville").text else None,
                "adresse": pdv.find("adresse").text.replace(",", " ").replace(";", " ") if pdv.find("adresse") is not None and pdv.find("adresse").text else None,
                "nom": p.get("nom"),
                "maj": p.get("maj"),
                "valeur": p.get("valeur")
            }
            data.append(row)
    df = pd.DataFrame(data)
    
    ###### CLEAN DF AND ADD TYPES ##############
    
    print(df.dtypes)
    
    df["id"] = df["id"].astype(int)
    df["latitude"] = df["latitude"].replace(["", "0"], np.nan).astype(float)
    df["longitude"] = df["longitude"].replace(["", "0"], np.nan).astype(float)
    df['adresse'] = df['adresse'].str.replace("\n", " ", regex=True)
    df["valeur"] = df["valeur"].astype(float)
    
    # (le cp est de valeur '35***' sur une des valeurs de 2008 et son id "35200004" ne correspond a aucune station sur d'autres année, donc on supprime.)
    df = df[df["cp"] != "35***"]
    df["cp"] = df["cp"].replace("", np.nan).astype(int)

    
    # (format date used between 2014 and now)
    df['maj_without_microsec_with_T'] = pd.to_datetime(df['maj'], format='%Y-%m-%dT%H:%M:%S', errors='coerce').dt.strftime('%Y_%m_%d_%H:%M')
    # (format date used between 2007 and 2013)
    df['maj_without_microsec'] = pd.to_datetime(df['maj'], format='%Y-%m-%d %H:%M:%S', errors='coerce').dt.strftime('%Y_%m_%d_%H:%M')
    df['maj_with_microsec'] = pd.to_datetime(df['maj'], format='%Y-%m-%d %H:%M:%S.%f', errors='coerce').dt.strftime('%Y_%m_%d_%H:%M')
    
    df['maj'] = df['maj_with_microsec'].fillna(df['maj_without_microsec']).fillna(df['maj_without_microsec_with_T'])
    df = df.drop(columns=["maj_without_microsec", "maj_with_microsec", "maj_without_microsec_with_T"])

    # values format used between 2007 and 2021 = 500 to 2000
    # values format used between 2022 ans now = 0.5 to 2.0 (need to be adapted to 500 to 2000)
    df['valeur'] = df['valeur'] * 1000 if int(year) > 2021 else df['valeur']
    
    print(df.dtypes)
    
    ########### SAVE DF TO CSV ###################
    
    os.makedirs("results/inputs_csv", exist_ok=True)
    df.to_csv(f"results/inputs_csv/PrixCarburants_annuel_{year}.csv", index=False)
    print(df.head(5))

LOAD 2007
<Response [200]>
PrixCarburants_annuel_2007.xml
PrixCarburants_annuel_2007.xml extracted
id           object
latitude     object
longitude    object
cp           object
ville        object
adresse      object
nom          object
maj          object
valeur       object
dtype: object
id             int32
latitude     float64
longitude    float64
cp             int32
ville         object
adresse       object
nom           object
maj           object
valeur       float64
dtype: object
        id   latitude  longitude    cp                  ville  \
0  1000001  4620114.0   519791.0  1000  SAINT-DENIS-LèS-BOURG   
1  1000001  4620114.0   519791.0  1000  SAINT-DENIS-LèS-BOURG   
2  1000001  4620114.0   519791.0  1000  SAINT-DENIS-LèS-BOURG   
3  1000001  4620114.0   519791.0  1000  SAINT-DENIS-LèS-BOURG   
4  1000001  4620114.0   519791.0  1000  SAINT-DENIS-LèS-BOURG   

           adresse     nom               maj  valeur  
0  ROUTE NATIONALE  Gazole  2007_01_04_18:43   999.0  
1  

<Response [200]>
PrixCarburants_annuel_2014.xml
PrixCarburants_annuel_2014.xml extracted
id           object
latitude     object
longitude    object
cp           object
ville        object
adresse      object
nom          object
maj          object
valeur       object
dtype: object
id             int32
latitude     float64
longitude    float64
cp             int32
ville         object
adresse       object
nom           object
maj           object
valeur       float64
dtype: object
        id   latitude  longitude    cp                  ville  \
0  1000001  4620114.0   519791.0  1000  SAINT-DENIS-LèS-BOURG   
1  1000001  4620114.0   519791.0  1000  SAINT-DENIS-LèS-BOURG   
2  1000001  4620114.0   519791.0  1000  SAINT-DENIS-LèS-BOURG   
3  1000001  4620114.0   519791.0  1000  SAINT-DENIS-LèS-BOURG   
4  1000001  4620114.0   519791.0  1000  SAINT-DENIS-LèS-BOURG   

           adresse     nom               maj  valeur  
0  ROUTE NATIONALE  Gazole  2014_01_02_11:08  1304.0  
1  ROUTE NATI

<Response [200]>
PrixCarburants_annuel_2021.xml
PrixCarburants_annuel_2021.xml extracted
id           object
latitude     object
longitude    object
cp           object
ville        object
adresse      object
nom          object
maj          object
valeur       object
dtype: object
id             int32
latitude     float64
longitude    float64
cp             int32
ville         object
adresse       object
nom           object
maj           object
valeur       float64
dtype: object
        id   latitude  longitude    cp                  ville  \
0  1000001  4620114.0   519791.0  1000  SAINT-DENIS-LèS-BOURG   
1  1000001  4620114.0   519791.0  1000  SAINT-DENIS-LèS-BOURG   
2  1000001  4620114.0   519791.0  1000  SAINT-DENIS-LèS-BOURG   
3  1000001  4620114.0   519791.0  1000  SAINT-DENIS-LèS-BOURG   
4  1000001  4620114.0   519791.0  1000  SAINT-DENIS-LèS-BOURG   

                 adresse     nom               maj  valeur  
0  596 AVENUE DE TREVOUX  Gazole  2021_01_02_15:13  1252.0  
1

### Analyses

In [3]:
# print(f"ANALYSE {year}")

In [4]:
# # check number of staions (id unique) and numbers of records
# print("id unique", len(df["id"].unique()))

# print("all values", len(df))

In [5]:
# check null values for all columns
# df.isnull().sum()

In [6]:
# #check bad string
# print(df[df['adresse'].str.startswith("ZI La Laure", na=False)])

In [7]:
# # check what ID have None data
# df2 = df.copy()
# df2 = df2[df2['adresse'].isna()]

# df2 = df2[["id", "adresse"]].drop_duplicates()
# df2.head(25)